## Pull B1610 Data from Elexon API

In this cell we:

1. Import and configure the Elexon API client.  
2. Define the time window for which we want to pull B1610 data.  
3. Retrieve the data and display the first 3 rows.

In [4]:
import sys, os
# ensure the workspace root is on PYTHONPATH so src.config can be imported
sys.path.insert(0, os.path.abspath('..'))

In [6]:
from src.config import ELEXON_API_KEY, ELEXON_BASE_URL
from src.fetching.elexon_client import ElexonApiClient
import requests

client = ElexonApiClient(ELEXON_API_KEY)

start_date = '2023-10-01'
end_date = '2023-10-31'

response = requests.get(
	f'{ELEXON_BASE_URL}/demand/actual/total',
	params={'start_date': start_date, 'end_date': end_date}
)

print(response.text)

{ "statusCode": 404, "message": "Resource not found" }
